In [2]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName('logreg').getOrCreate()

In [4]:
from pyspark.ml.classification import LogisticRegression

In [21]:
input_file_path="file:///C:/Users/ckp43_000/Documents/Admission_Predict.csv"

In [28]:
data=spark.read.csv(input_file_path,inferSchema=True,header=True)

In [29]:
data.show(10)

+----------+---------+-----------+-----------------+---+----+----+--------+----------------+
|Serial No.|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+----------+---------+-----------+-----------------+---+----+----+--------+----------------+
|         1|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|
|         2|      324|        107|                4|4.0| 4.5|8.87|       1|            0.76|
|         3|      316|        104|                3|3.0| 3.5| 8.0|       1|            0.72|
|         4|      322|        110|                3|3.5| 2.5|8.67|       1|             0.8|
|         5|      314|        103|                2|2.0| 3.0|8.21|       0|            0.65|
|         6|      330|        115|                5|4.5| 3.0|9.34|       1|             0.9|
|         7|      321|        109|                3|3.0| 4.0| 8.2|       1|            0.75|
|         8|      308|        101|                2|3.0| 4.0| 7.9|    

In [30]:
data.count()

400

In [34]:
data.columns

['Serial No.',
 'GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR ',
 'CGPA',
 'Research',
 'Chance of Admit ']

In [31]:
from pyspark.ml.linalg import Vector

In [32]:
from pyspark.ml.feature import VectorAssembler

In [35]:
assembler=VectorAssembler(inputCols=['GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR ',
 'CGPA'],outputCol='features')

In [38]:
output=assembler.transform(data)

In [39]:
output.show(10)

+----------+---------+-----------+-----------------+---+----+----+--------+----------------+--------------------+
|Serial No.|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |            features|
+----------+---------+-----------+-----------------+---+----+----+--------+----------------+--------------------+
|         1|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|[337.0,118.0,4.0,...|
|         2|      324|        107|                4|4.0| 4.5|8.87|       1|            0.76|[324.0,107.0,4.0,...|
|         3|      316|        104|                3|3.0| 3.5| 8.0|       1|            0.72|[316.0,104.0,3.0,...|
|         4|      322|        110|                3|3.5| 2.5|8.67|       1|             0.8|[322.0,110.0,3.0,...|
|         5|      314|        103|                2|2.0| 3.0|8.21|       0|            0.65|[314.0,103.0,2.0,...|
|         6|      330|        115|                5|4.5| 3.0|9.34|       1|             

In [ ]:
my_data.withColumnRenamed()

In [42]:
my_data=output.select('Research','features')

In [48]:
new_my_data=my_data.withColumnRenamed('Research','label')

In [49]:
new_my_data.show(10)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    1|[337.0,118.0,4.0,...|
|    1|[324.0,107.0,4.0,...|
|    1|[316.0,104.0,3.0,...|
|    1|[322.0,110.0,3.0,...|
|    0|[314.0,103.0,2.0,...|
|    1|[330.0,115.0,5.0,...|
|    1|[321.0,109.0,3.0,...|
|    0|[308.0,101.0,2.0,...|
|    0|[302.0,102.0,1.0,...|
|    0|[323.0,108.0,3.0,...|
+-----+--------------------+
only showing top 10 rows



In [52]:
from pyspark.ml.classification import LogisticRegression

In [53]:
my_log_reg_model=LogisticRegression()

In [54]:
fitted_logreg=mylogreg_model.fit(new_my_data)

In [58]:
log_summary=fitted_logreg.summary

In [60]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [61]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  1.0|[337.0,118.0,4.0,...|[-3.2639831297014...|[0.03682766121973...|       1.0|
|  1.0|[324.0,107.0,4.0,...|[-1.6532646432421...|[0.16066821172423...|       1.0|
|  1.0|[316.0,104.0,3.0,...|[-0.0199491879687...|[0.49501286840082...|       1.0|
|  1.0|[322.0,110.0,3.0,...|[-0.7779682951811...|[0.31475793006685...|       1.0|
|  0.0|[314.0,103.0,2.0,...|[0.55834850934115...|[0.63607033116586...|       0.0|
|  1.0|[330.0,115.0,5.0,...|[-2.2887475578452...|[0.09205918094496...|       1.0|
|  1.0|[321.0,109.0,3.0,...|[-0.5873164418108...|[0.35725082377669...|       1.0|
|  0.0|[308.0,101.0,2.0,...|[0.91063080243332...|[0.71312922709578...|       0.0|
|  0.0|[302.0,102.0,1.0,...|[2.33724997857842...|[0.91191543793503...|       0.0|
|  0.0|[323.0,10

In [62]:
 lr_train,lr_test=new_my_data.randomSplit([0.7,0.3])

In [63]:
final_model=LogisticRegression()

In [64]:
fit_final=final_model.fit(lr_train)

In [65]:
prediction_and_labels=fit_final.evaluate(lr_test)

In [69]:
prediction_and_labels.predictions.show(10)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|[295.0,101.0,2.0,...|[3.37965919483598...|[0.96706275147193...|       0.0|
|    0|[296.0,99.0,2.0,2...|[2.99979531300360...|[0.95256487889098...|       0.0|
|    0|[296.0,99.0,2.0,3...|[2.56926878075231...|[0.92885739111938...|       0.0|
|    0|[296.0,101.0,1.0,...|[3.06622691871024...|[0.95547794167440...|       0.0|
|    0|[297.0,98.0,2.0,2...|[2.66256137164786...|[0.93478099584926...|       0.0|
|    0|[298.0,92.0,1.0,2...|[2.72517009173985...|[0.93849563700100...|       0.0|
|    0|[298.0,99.0,2.0,4...|[2.38605917092676...|[0.91575804956826...|       0.0|
|    0|[299.0,94.0,1.0,1...|[3.24182931854944...|[0.96237839824647...|       0.0|
|    0|[300.0,102.0,2.0,...|[3.00825850394143...|[0.95294582707094...|       0.0|
|    0|[300.0,10

In [70]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator,
                                   MulticlassClassificationEvaluator)

In [71]:
my_eval=BinaryClassificationEvaluator()

In [72]:
my_final_roc=my_eval.evaluate(prediction_and_labels.predictions)

In [73]:
my_final_roc

0.8321724709784412